In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [4]:
 # create engine to hawaii.sqlite
# inspector = inspect(engine)
# inspect(engine).get_table_names()
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [5]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [6]:
# View the classes
Base.classes.keys()

['measurement', 'station']

In [7]:
# View all of the classes that automap found

Measurement=Base.classes.measurement
Station=Base.classes.station

In [8]:
# Save references to each table
inspector = inspect(engine)
columns = inspector.get_columns('measurement')
for column in columns:
    print(f'{column["name"]}      \t{column["type"]}')

id      	INTEGER
station      	TEXT
date      	TEXT
prcp      	FLOAT
tobs      	FLOAT


In [9]:
inspector = inspect(engine)
columns = inspector.get_columns('station')
for column in columns:
    print(f'{column["name"]}      \t{column["type"]}')

id      	INTEGER
station      	TEXT
name      	TEXT
latitude      	FLOAT
longitude      	FLOAT
elevation      	FLOAT


In [10]:
# Create our session (link) from Python to the DB
session=Session(engine)

In [14]:
station.query(func.count(station.id)).all()

NameError: name 'station' is not defined

In [ ]:
session.query(func.count(measurement.station)).all()

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.tab
engine.execute('SELECT max(date)FROM measurement').fetchall()

In [ ]:
prcp_data=session.query(Measurement.date, Measurement.prcp).filter(Measurement.date > "2016-08-23").\
    filter(Measurement.date <= "2017-08-23").all()

prcp_data

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.

year_ago = dt.date(2017,8,23)-dt.timedelta(days=365)

# Perform a query to retrieve the data and precipitation scores
prcp_data=session.query(Measurement.date, Measurement.prcp).filter(Measurement.date > "2016-08-23").\
    filter(Measurement.date <= "2017-08-23").all()

# Save the query results as a Pandas DataFrame and set the index to the date column
prcp_data = pd.DataFrame(prcp_data).set_index('date')

# Sort the dataframe by date
prcp_data.sort_values(by='date', ascending=True)
prcp_data=prcp_data.dropna()

# Use Pandas Plotting with Matplotlib to plot the data

prcp_data.plot(rot=60)

plt.legend(title='Rain amounts', fontsize=12)
plt.title('2016-2017 Precipitation', fontsize=18)
plt.show()



In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
prcp_data.describe()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
total_stations = session.query(Station.id).count() 
print(f'There are {total_stations} stations in the dateset.')

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
active_stations = session.query(Measurement.station, func.count(Measurement.station)).\
group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()
active_stations

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
station_number = 'USC00519281'
session.query(func.min(Measurement.tobs),
              func.max(Measurement.tobs),
              func.avg(Measurement.tobs)).filter(Measurement.station == station_number).all()

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
most_active_station = active_stations[0][0]

results = session.query(Measurement.date, Measurement.tobs).\
                        filter(Measurement.date > "2016-08-23").\
                        filter(Measurement.date <= "2017-08-23").\
                        filter(Measurement.station == most_active_station).all()
temp_observations = pd.DataFrame(results)

temp_observations.plot.hist(bins=12)

# Close session

In [ ]:
# Close Session
session.close()